In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['GROQ_API_KEY']=os.getenv('GROQ_API_KEY')

In [37]:
import base64

def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')


image_path = r"C:\Users\panka\OneDrive\Desktop\Cograd\Answersheet_Analysis\answer_sheet_1.jpg"


base64_image = encode_image(image_path)


In [51]:
prompt = f"""
Analyse precisely and do not hellusinate.Give question number and their Marks obtained in json format.consider that Question numbsr only which has Obtained marks handwritten,.Do not wite null or anything just laeve that .Give output in json format
ignore total marks section
"""

In [62]:
from groq import Groq

client = Groq()
def chat_completion(prompt,base64_image):
 return client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}",
                    },
                },
            ],
        }
    ],
    model="llama-3.2-90b-vision-preview",
    max_tokens=1000,
    temperature=0.1
)

answer=chat_completion(prompt=prompt,base64_image=base64_image).choices[0].message.content
print(answer)

**Question Number and Marks Obtained in JSON Format**

Based on the provided image, the following questions have handwritten marks:

*   Question 1: 10
*   Question 2: 2
*   Question 3: 3

The JSON format for these questions is as follows:

```json
{
    "question1": 10,
    "question2": 2,
    "question3": 3
}
```

This output only includes the questions with handwritten marks, excluding any null or empty values.


#test paper

In [63]:
image_path_Question_Paper= r"C:\Users\panka\OneDrive\Desktop\Cograd\Answersheet_Analysis\test_paper_1.jpg"
base64_image_Question_Paper= encode_image(image_path)

In [70]:
prompt_Question_Paper = f"""
Do not consider any answer sheet just current image.Give question number ,  questions and theirs max marks in json format if a  question has more than one subpart then give  question as their subpart but max maarks of combined.Give output in json format
"""

In [71]:
question=chat_completion(prompt=prompt_Question_Paper,base64_image=base64_image_Question_Paper).choices[0].message.content
print(question)

Here is the output in JSON format:

```
[
  {
    "question_number": 1,
    "question": "Question 1",
    "max_marks": 7
  },
  {
    "question_number": 2,
    "question": "Question 2",
    "max_marks": 7
  },
  {
    "question_number": 3,
    "question": "Question 3",
    "max_marks": 7
  }
]
```

Note: Since there are only three questions on the answer sheet, the output only includes these three questions. If there were more questions, they would be included in the output as well.


In [72]:
#Comnibe both

In [75]:
#few sort prompting for model
outputs="""

{
  "questions": [
    {
      "question_number": "Q1",
      "description": "Instruction execution cycle",
      "max_marks": 5,
      "marks_obtained": 2
    }
  ]
}
"""


prompt_combining_ocr=f"""
  You are given two JSON objects: {answer}` and :{question}.

- {question} contains a list of questions, subparts, descriptions, and maximum marks.
- {answer}contains the marks obtained for each question.

Your task is to merge these two JSONs and give output only json object.

Return a **single valid JSON object** with all questions, subparts, `"max_marks"`, and `"marks_obtained"`.

yout output will be like {outputs} .you will give just output not code


"""

In [77]:
def chat_completion(prompt):
 return client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
            ],
        }
    ],
    model="llama-3.2-90b-vision-preview",
    max_tokens=1000,
    temperature=0.1
)

combined_output=chat_completion(prompt=prompt_combining_ocr).choices[0].message.content
print(combined_output)

```json
{
  "questions": [
    {
      "question_number": "Q1",
      "description": "Question 1",
      "max_marks": 7,
      "marks_obtained": 10
    },
    {
      "question_number": "Q2",
      "description": "Question 2",
      "max_marks": 7,
      "marks_obtained": 2
    },
    {
      "question_number": "Q3",
      "description": "Question 3",
      "max_marks": 7,
      "marks_obtained": 3
    }
  ]
}
```


In [78]:
#Final_output

In [81]:
prompt_final=f"just tell me the topic and araange them according to obtained marks in ascending order considering this {combined_output}"

In [82]:
def chat_completion(prompt):
 return client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
            ],
        }
    ],
    model="llama-3.2-90b-vision-preview",
    max_tokens=1000,
    temperature=0.0
)

In [83]:
final_answer=chat_completion(prompt=prompt_final).choices[0].message.content
print(final_answer)

Topic: Question Analysis

Questions in ascending order of marks obtained:

1. Q2 - Question 2 (2 marks)
2. Q3 - Question 3 (3 marks)
3. Q1 - Question 1 (10 marks)


In [24]:
!pip install opencv-python pytesseract numpy


  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\panka\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [31]:
import cv2
import pytesseract
import re

# Set Tesseract path if not added to PATH
# pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

def preprocess_image(image_path):
    """Preprocess image for better OCR performance"""
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    return img

def extract_marks(image_path):
    """Extract marks from an answer sheet"""
    img = preprocess_image(image_path)
    text = pytesseract.image_to_string(img, config="--psm 6")  # OCR extraction

    # Use regex to find marks (e.g., 7/10, 8.5, 9, 10/10)
    marks = re.findall(r'\b\d{1,2}(?:[./]\d{1,2})?\b', text)

    return marks

# Example Usage
image_path = "test_paper_1.jpg"  # Update with the correct path
marks_list = extract_marks(image_path)

print("Extracted Marks:", marks_list)


ModuleNotFoundError: No module named 'cv2'